In [6]:
# ===== セル1: ライブラリのインポート =====
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt

import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cpu


In [10]:
# ===== セル2: データの前処理と読み込み =====
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# フォルダ構成:
# dataset/
# ├── train/
# │   ├── cat/
# │   └── dog/
# └── val/
#     ├── cat/
#     └── dog/

train_dataset = ImageFolder('dataset/train', transform=transform)
val_dataset = ImageFolder('dataset/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print("Classes:", train_dataset.classes)


FileNotFoundError: Found no valid file for the classes cat. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
# ===== セル3: モデルの定義 =====
model = resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2クラス分類
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# ===== セル4: 学習ループ =====
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')


In [ ]:
# ===== セル5: 画像の分類関数 =====
def predict_image(image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return train_dataset.classes[predicted.item()]


In [ ]:
# ===== セル6: 画像ファイルで推論してみる =====
test_image_path = 'dataset/val/cat/cat1.jpg'  # ← 例: 適切な画像パスに変更
result = predict_image(test_image_path)
print("予測結果:", result)
